In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import ast
import json
import os

# Data Preparation

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/University/Research/SemEval 2025: Task 7
# %cd /content/drive/MyDrive/Research/SemEval 2025: Task 7

/content/drive/MyDrive/University/Research/SemEval 2025: Task 7


  Data Properties:

      posts_df:
          posts['text'] is either a tuple of 3 or an empty string
          posts['ocr'] is a list of tuples of 3 which is the ocr for each picture

      fact_checks_df:
          fact_checks_df['title'] is either a tuple of 3 or an empty string
          fact_checks_df['claim'] is always a tuple of 3

      mapping_df:
          nothing especial

In [5]:
parse_col = lambda s: ast.literal_eval(s.replace('\n', '\\n')) if s else s

fact_checks_df = pd.read_csv('./data/cleaned data/fact_checks.csv').fillna('').set_index('fact_check_id')
# fact_checks_df = pd.read_csv('./data/cleaned data/fact_checks_test.csv').fillna('').set_index('fact_check_id')

for col in ['claim', 'title']:
    fact_checks_df[col] = fact_checks_df[col].apply(parse_col)


posts_df = pd.read_csv('./data/cleaned data/posts.csv').fillna('').set_index('post_id')
# posts_df = pd.read_csv('./data/cleaned data/posts_test.csv').fillna('').set_index('post_id')


mapping_df = pd.read_csv('./data/original data/pairs.csv')

with open('./data/original data/tasks.json', 'r') as file:
    tasks = json.load(file)

In [6]:
def sort_df(df, column, function, ascending = False):
    df_sorted = df.copy()
    df_sorted['text_length'] = df_sorted[column].apply(function)
    df_sorted = df_sorted.sort_values(by='text_length', ascending=ascending)
    return df_sorted

# preparing fact_checks

In [ ]:
fact_checks_df['content'] = fact_checks_df.apply(lambda r: r['claim'][0], axis=1)

In [ ]:
fc_sorted = sort_df(fact_checks_df, 'content', len)
fc_sorted

,claim,instances,title,language,content,text_length
fact_check_id,,,,,,
9136,"(18 de novembro de 2016 Maurício Moraes, Étore...","[(1479427140.0, 'https://apublica.org/checagem...",(5 deslizes de Temer ao falar do Congresso no ...,por,"18 de novembro de 2016 Maurício Moraes, Étore ...",9647
100280,(Papa Francisco Condenado por Tráfico de Crian...,"[(1571270280.0, 'https://www.boatos.org/religi...",(Papa Francisco é condenado por tráfico de cri...,por,Papa Francisco Condenado por Tráfico de Crianç...,7610
9135,"(18 de novembro de 2016 Maurício Moraes, Étore...","[(1479427140.0, 'https://apublica.org/checagem...",(5 deslizes de Temer ao falar do Congresso no ...,por,"18 de novembro de 2016 Maurício Moraes, Étore ...",5582
36946,(Checamos frases de seis senadores no debate q...,"[(1456444740.0, 'https://apublica.org/checagem...","(6 blefes e meias-verdades sobre o pré-sal, 6 ...",por,Checamos frases de seis senadores no debate qu...,4638
146978,(Von PS - 18. Oktober 2018 7 3446 Share Immer ...,"[(1540545597.0, 'https://correctiv.org/fakten-...",(DSGVO – Namen an Klingelschildern sind zuläss...,deu,Von PS - 18. Oktober 2018 7 3446 Share Immer d...,4169
...,...,...,...,...,...,...
171404,"(صورة من, Photo from, [(ara, 1.0)])","[(1600127880.0, 'https://fatabyyano.net/%d9%87...",(هذه الصورة ليس من فيضانات السودان الأخيرة عام...,ara,صورة من,7
8523,"(+18, +18, [(ara, 1.0)])","[(None, 'https://dabegad.com/%d8%ad%d9%82%d9%8...",(حقيقة فيديو كيفية استخدام الجراب لشحن الموباي...,ara,+18,3
7726,"(3>, 3>, [(ara, 1.0)])","[(None, 'https://dabegad.com/%d8%ad%d9%82%d9%8...",(حقيقة هاشتاج ‫#‏بنحب_الرئيس‬ يتصدر تويتر ويتخ...,ara,3>,2


In [ ]:
fc_sorted.head(8000)['content'].str.split().str.len().sum()

389662

In [ ]:
# fc_sorted.loc[mapping_df['fact_check_id'].to_list()].sort_values(by='text_length', ascending=False)

In [ ]:
fc_sorted['text_length'].iloc[3000]

273

In [ ]:
fc_sorted['content'].iloc[3000]

'Ein Bild auf Facebook zeigt vermeintlich ein Schild, das an einem Spielplatz in Hannover stehen soll. Darauf steht: "Spielplatz für Kinder und Kinderinnen." Dazu ist das Stadt-Logo und die Anschrift des zuständigen Fachbereichs für Umwelt und Stadtgrün in Hannover zu lesen'

# preparing posts

In [7]:
posts_sorted = sort_df(posts_df, 'content', len)
posts_sorted

,ocr,text,content,eng_content,language,text_length
post_id,,,,,,
25778,AFTONAL LIST OF AVERE EVENTS OF SPECIAL p Pfiz...,Neun Seiten Nebenwirkungen: Komplette Auflistu...,AFTONAL LIST OF AVERE EVENTS OF SPECIAL p Pfiz...,AFTONAL LIST OF AVERE EVENTS OF SPECIAL p Pfiz...,deu,32243
14526,"hypothyroidism; Autoimmune er ear disease,Auto...",While they are distracted with Ukraine and Rus...,"hypothyroidism; Autoimmune er ear disease,Auto...","hypothyroidism; Autoimmune er ear disease,Auto...",eng,24760
26466,sina Sina Finance Hong Kong Stocks > Hong Kong...,"The cat crying and the mouse fake mercy, the g...",sina Sina Finance Hong Kong Stocks > Hong Kong...,sina Sina Finance Hong Kong Stocks > Hong Kong...,eng,22249
24616,Covid-190 home Mr Gree100 - - - Besli sadiste ...,Corona beta is put on the streets as their own...,Covid-190 home Mr Gree100 - - - Besli sadiste ...,Covid-190 home Mr Gree100 - - - Besli sadiste ...,eng,20770
18820,,Sé lo que son informes epidemiológicos; los do...,Sé lo que son informes epidemiológicos; los do...,I know what epidemiological reports are; the t...,spa,19851
...,...,...,...,...,...,...
21864,,,,,eng,0
21860,,,,,eng,0
21914,,,,,eng,0


In [8]:
dev_posts_ids = []
dev_posts_ids.extend(tasks['crosslingual']['posts_dev'])
for lang in tasks['monolingual'].keys():
    dev_posts_ids.extend(tasks['monolingual'][lang]['posts_dev'])

In [9]:
len(dev_posts_ids)

2443

In [10]:
posts_sorted_dev = posts_sorted.loc[dev_posts_ids].sort_values(by='text_length', ascending=False)

posts_sorted_dev

,ocr,text,content,eng_content,language,text_length
post_id,,,,,,
18820,,Sé lo que son informes epidemiológicos; los do...,Sé lo que son informes epidemiológicos; los do...,I know what epidemiological reports are; the t...,spa,19851
7625,1550 DECRETO NÚMERO DE 2020 PÁGINA 20 DE 24 Co...,Cierre de Bares y Discotecas en todo el territ...,1550 DECRETO NÚMERO DE 2020 PÁGINA 20 DE 24 Co...,1550 DECREE NUMBER OF 2020 PAGE 20 OF 24 Conti...,spa,8728
27219,Is he planing to bring the Anunnaki back to Ea...,"CERN LHC - The Apparatus of Osiris, Shiva's St...",Is he planing to bring the Anunnaki back to Ea...,Is he planing to bring the Anunnaki back to Ea...,eng,6885
12941,,L'IGNORANCE VA NOUS TUER!!! CONSOMMATION DES F...,L'IGNORANCE VA NOUS TUER!!! CONSOMMATION DES F...,IGNORANCE WILL KILL US!!! FASTING FRUIT CONSUM...,fra,6822
8903,EL DR. FAUCI APROBÓ LA HIDROXICLOROQUINA HACE ...,EL DR. FAUCI APROBÓ LA HIDROXICLOROQUINA HACE ...,EL DR. FAUCI APROBÓ LA HIDROXICLOROQUINA HACE ...,THE DOCTOR. FAUCI APPROVED THE HYDROXYCHLOROQU...,spa,6514
...,...,...,...,...,...,...
3787,,is breaking apart,is breaking apart,is breaking apart,eng,17
26471,,踢检疫犬的那个人出来，我保证不打你,踢检疫犬的那个人出来，我保证不打你,When the person who kicked the quarantine dog ...,zho,17
5927,,Anyone?,Anyone?,Anyone?,eng,7


In [11]:
posts_sorted_dev['content'].str.split().str.len().sum()

199433

In [12]:
posts_df[posts_df['language'].apply(lambda lang: lang == 'tha')].head(2)['content'].to_list()

["ือจะเป็นจริง ภาพนี้ถูกทำนายไว้เมื่อ 60 ปีที่แล้ว ปกนิตยสารในอิตาลีที่ตีพิมพ์ในปี 2505 โดยบอกว่านี่คือภาพของโลก ในปี 2565 (อีก 2 ปี) บรรยายภาพ คนอยู่บนรถที่มีครอบแก้วปิด ป้องกันมลพิษและเชื้อโรค DOMENICA DEL CORRIERE grande di lul Nova Scotia 115-13 RISI FRIBEY'S MOSER 50 FORTON WITH YOU FOR EVER It IMEI 1 pers 60 In città gireremo così? proble p",
 '1. true Messenger .ill 4G ครับ  มนุษย์โลกถ้าจะถึงจุดจบแล้วครับ 21 19:47 19:29 เปรมปิติ ชูใจใบบัว 42 นาที • @ ตะกี้นี้ใครหันแสงสีฟ้า พุ่งผ่านเหนือหัวทางทิศเหนือพ่อง ว้าว 9+ Rung Wandee Pia amat 6 นาทีที่แล้ว 28% 14 แสดงความคิดเห็น 29% ความคิดเห็น 14 รายการ › แชร์ 9 =']

In [13]:
posts_df[posts_df['language'].apply(lambda lang: lang == 'tha')].head(2)['eng_content'].to_list()

["will it be true This picture was predicted in the 60's. last year Cover of a magazine in Italy published in 1962. By saying that this is a picture of the world in 2022. (2 more years) Image caption A man sits in a car with a closed glass cover. prevent pollution and germs DOMENICA DEL CORRIERE grande di lul Nova Scotia 115-13 RISI FRIBEY'S MOSER 50 FORTON WITH YOU FOR EVER It IMEI 1pers 60 In città gireremo così? problem p",
 "1. true Messenger .ill 4G yes  District, the world's man, if it reaches the end 21 19:47 19:29 Prempiti Choojai Baibua 42 minutes • @ Just now, who turned the blue light? rushed over the head to the north wow 9+ Rung Wandee Pia amat 6 minutes ago 28% 14 comment 29% 14 comments > share 9 ="]

# mT5-multilingual-XLSum

In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [ ]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = [
                """
                23:58 = TVA Nouvelles <<AVERTISSEMENT POUR TOUS LES PARENTS AYANT DES ENFANTS QUI ONT DES APPAREILS ÉLECTRONIQUES, EX : IPOD, TABLETTES ETC.... IL YA UNE APPLICATION QUI S'APPELE TALKING ANGELA QUI POSE DES QUESTIONS AUX ENFANTS COMME: LEURS NOMS, OÙ ILS HABITTENT, NOMS DE LEURS ECOLES ET PEUT MÊME PRENDRE DES PHOTOS DE LEURS VISAGES EN APPUYANT SUR LE COEUR ROSE SITUÉ AU COIN INFÉRIEUR GAUCHE SANS QUE LES ENFANTS LE REALISENT. CETTE APPLICATION EST CRÉE PAR DES PEDOPHILES. S'IL VOUS PLAÎT VÉRIFIEZ LES TABLETTES DE VOS ENFANTS POUR S'ASSURER QU'ILS N'ONT PAS CETTE APP!!! S'IL VOUS PLAÎT TRANSMETTRE CE MESSAGE À VOS AMIS ET LES MEMBRES DE LA FAMILLE QUI ONT DES ENFANTS!!!>> = ||| Q 61% O م", '23:58
                """,
                """
                VAT News <<WARNING FOR ALL PARENTS WITH CHILDREN WHO HAVE ELECTRONIC DEVICES, EX: IPOD, TABLETS ETC.... THERE IS A APPLICATION CALLED TALKING ANGELA ASKING QUESTIONS TO CHILDREN LIKE: THEIR NAMES, WHERE THEY INHABIT, NAMES OF THEIR SCHOOLS AND CAN EVEN TAKE PHOTOS OF THEIR FACES BY PRESSING THE PINK HEART LOCATED IN THE LOWER CORNER LEFT WITHOUT THE CHILDREN REALIZE. THIS APP IS CREATED BY PEDOPHILES. IF YOU PLEASE CHECK YOUR TABLETS CHILDREN TO MAKE SURE THEY HAVE NO NOT THIS APP!!! PLEASE FORWARD THIS MESSAGE TO YOUR FRIENDS AND FAMILY MEMBERS WHO HAVE CHILDREN!!!>> = ||| Q 61% O م
                """
                ]

input_ids = tokenizer(
    [WHITESPACE_HANDLER(text) for text in article_text],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=150,
    min_length=30,
    no_repeat_ngram_size=2,
    num_beams=4,
    length_penalty=1.2
)

summaries = [
    tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    for output in output_ids
]

# Print summaries for all inputs
for i, summary in enumerate(summaries):
    print(f"Summary {i + 1}: {summary}")


Summary 1: Plus de 500 millions d'euros sont nécessaires pour la lutte contre le coronavirus.


# bart-large-cnn

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
tmp = summarizer(article_text, max_length=90, min_length=30, do_sample=False)
tmp

Your max_length is set to 90, but your input_length is only 43. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


[{'summary_text': "WARNING !! A new thing circulating now. People are going door to door handing out masks. They say it's a new initiative from local government. It's doused with chemicals which knocks you out cold. They then rob you!! Please DO NOT accept masks from strangers."},
 {'summary_text': '"He\'s a damn sight better than any of the other Presidents that have been there before him, including those that you admire, Miss Ardern"'}]

In [ ]:
tmp[1]['summary_text']

'WARNING FOR ALL PARENTS WITH CHILDREN WHO HAVE ELECTRONIC DEVICES, EX: IPOD, TABLETS ETC.... There is a APPLICATION CALLED TALKING ANGELA ASKING QUESTIONS TO CHilDREN. This APP is CREATED BY PEDOPHILES. IF YOU PLEASE CHECK YOUR TABLETS CHIL DREN TO MAKE SURE THEY HAVE NO NOT THIS APP!!! PLEASE FORWARD THIS MESSAGE TO YOUR FRIENDS AND FAMILY MEMBERS WHO HAVE CHILFREN!!!'

# Falcon3-7B-Instruct

In [ ]:
model_2 = pipeline("summarization", model="tiiuae/Falcon3-7B-Instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
tmp = model_2(article_text, max_length=90, min_length=30, do_sample=False)
tmp

Your max_length is set to 90, but your input_length is only 47. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)


[{'summary_text': '"I voted for you to please put it on to see if you don\'t want to get rid of the caution you handing out..."'},
 {'summary_text': '"Actually, he\'s a damn sight better than any of the other Presidents that were there before him, including those that you admire, Miss Ardern."'}]

# Qwen2.5-1.5B-Instruct

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# Qwen2.5-7B-Instruct

In [19]:
!pip install transformers bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "Qwen/Qwen2.5-7B-Instruct"

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# Building summaries

In [21]:
# Batch processing with the Hugging Face code structure
def generate_summaries_in_batches(texts, batch_size=4):
    summaries = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Processing Batches"):
        batch_texts = texts[i:i+batch_size]

        prompt = """
                    **Summarize the following social media text into a concise and factual summary. Your goal is to capture the essential information, focusing only on the main topic while omitting distracting details, emotions, personal opinions, and social media-specific elements. Ensure the summary is as brief and precise as possible.**

                    ---

                    ### **Guidelines:**

                    #### **Include**:
                    - Names of people, organizations, and landmarks.
                    - Locations (cities, countries, buildings, etc.).
                    - Dates, times, and chronological order if multiple events.
                    - Numerical data (e.g., statistics, monetary values, timelines).
                    - Professional titles, projects, and event details.
                    - Direct quotes **only if critical** to the main information.

                    #### **Exclude**:
                    - Descriptive language, emotions, or personal opinions.
                    - Social media-specific elements like hashtags, likes, mentions, and emojis.
                    - Background or contextual information unless essential.
                    - Gender-specific references unless directly relevant.
                    - Distracting or secondary details unrelated to the main topic.

                    #### **Format**:
                    1. Use active voice.
                    2. Present factual information only.
                    3. Keep the summary to a **single sentence (preferred)** or **maximum of 30 words**.
                    4. Prioritize the most critical detail first.
                    5. Maintain clarity and chronological order if multiple events are mentioned.
                    6. Give the summary only in English Language
                    7. If the input in empty or does not include enough infomation to summarize, just output the input without any change.

                    ---

                    ### **Task**:
                    Now, summarize the following social media text in English according to these guidelines:

                    **{input}**
        """

        messages = [
            [{"role": "system", "content": "You are a precise summarization tool that transforms social media content into concise, fact-focused summaries. Your objective is to maximize information density while maintaining perfect clarity."},
             {"role": "user", "content": prompt.format(input=text)}]
            for text in batch_texts
        ]
        batch_prompts = [
            tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)
            for message in messages
        ]
        model_inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)
        generated_ids = model.generate(**model_inputs, max_new_tokens=512)
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        batch_summaries = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        summaries.extend(batch_summaries)

    return summaries

## Single Prompts

In [ ]:
text = """
I REPUBLIQUE DU CAMEROUN Paix-Travail-Patrie ******* MINISTERE DE LA SANTE PUBLIQUE ********* SECRETARIAT GENERAL ********** DELEGATION REGIONALE DE LA SANTE PUBLIQUE DU CENTRE ********* SGI- COVID 19 ********* 2. AZITHROMYCINE 250mg: 06 CP 2 cp en prise unique le 1 jour, puis, 1 cp/jr du 2 au 5e jour. DANS LA MESURE DU POSSIBLE, AJOUTER : 3. VITAMINE C 1000mg: 10cp lep dans un demi vers d'eau le matin REPUBLIC OF CAMEROON Peace-Work-Fatherland ******* 4. ZINC 20 mg: 05 cp MINISTRY OF PUBLIC HEALTH ********* 1. PLAQUENIL (HYDROXYCHLOROQUINE) 200mg CP: 10CP I cp matin 1 ep soir, pendant 05 jours et chaque prise après un repas lep dans un demi vers d'eau le matin 5. Boire beaucoup de boissons chaudes GENERAL SECRETARIAT ********** PUBLIC HEALTH DELEGATION OF THE CENTER REGION PLAN DE MISE SOUS TRAITEMENT POUR UN PATIENT ADULTE DEPISTE COVID 19++ ********* COVID 19-IMS *********Vs  LE PROTOCOLE. COVID-19  Le virus à muté!!! Il peut se manifester autrement, ne vous fiez pas aux anciens symptômes . Vous pouvez avoir d'autres symptômes nouveaux tels que: - Surdité haut - La perte de cheveux - Le hoquet persistant SI VOUS RESSENTEZ UN MOINDRE MALAISE APPLIQUEZ D'ABORD LE PROTOCOLE AVANT DE CHERCHEZ A COMPRENDRE. NE SOYEZ PAS SPECTATEUR DE VOTRE PROTECTION PHYSIQUE ET MENTAL PRENEZ VOTRE PROTECTION EN MAIN!!! Tous Contre Coronavirus Minsante Infos Cameroun Malachie Manaouda Vs CoronaVirus  Jeunesse et Actions Sociales Serge Espoir Matomba Mairie Douala 5ème Mairie Douala 3 Nourane Foster President Paul Biya
"""

In [ ]:
prompt = f"""
            **Summarize the following social media text into a concise and factual summary. Your goal is to capture the essential information, focusing only on the main topic while omitting distracting details, emotions, personal opinions, and social media-specific elements. Ensure the summary is as brief and precise as possible.**

            ---

            ### **Guidelines:**

            #### **Include**:
            - Names of people, organizations, and landmarks.
            - Locations (cities, countries, buildings, etc.).
            - Dates, times, and chronological order if multiple events.
            - Numerical data (e.g., statistics, monetary values, timelines).
            - Professional titles, projects, and event details.
            - Direct quotes **only if critical** to the main information.

            #### **Exclude**:
            - Descriptive language, emotions, or personal opinions.
            - Social media-specific elements like hashtags, likes, mentions, and emojis.
            - Background or contextual information unless essential.
            - Gender-specific references unless directly relevant.
            - Distracting or secondary details unrelated to the main topic.

            #### **Format**:
            1. Use active voice.
            2. Present factual information only.
            3. Keep the summary to a **single sentence (preferred)** or **maximum of 30 words**.
            4. Prioritize the most critical detail first.
            5. Maintain clarity and chronological order if multiple events are mentioned.
            6. Give the summary in English Language
            ---

            ### **Task**:
            Now, summarize the following social media text according to these guidelines:

            **{text}**
"""
messages = [
    {"role": "system", "content": "You are a precise summarization tool that transforms social media content into concise, fact-focused summaries. Your objective is to maximize information density while maintaining perfect clarity."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

'The Ministry of Public Health in Cameroon recommends a treatment plan for adult COVID-19 patients including hydroxychloroquine, azithromycin, vitamin C, zinc, and fluids.'

In [ ]:
prompt = f"""
          Summarize the following input in between 10 to 30 words while preserving critical information, such as names, places, cities, dates, and key events.
          Exclude any details about the writer, including their emotions, opinions, gender, or other personal characteristics or any information about the structure of the input like the language of the text. Ensure the summary is concise, factual, and focused only on the main points.
          Avoid adding any information not present in the original input. Write the summary in consequence sentences and in English language.

          Here's the input:

          -----------------------------------------
          ""{text}""
          -----------------------------------------
      """
messages = [
    {"role": "system", "content": "You are a precise summarization tool that transforms social media content into concise, fact-focused summaries. Your objective is to maximize information density while maintaining perfect clarity."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

'The Ministry of Public Health in Cameroon recommends a treatment plan for an adult COVID-19 patient, including hydroxychloroquine, azithromycin, vitamin C, zinc, and hydration.'

## Summarize fact checks

In [ ]:
tmp = fc_sorted.sample(5)
tmp

,claim,instances,title,language,content,text_length
fact_check_id,,,,,,
333674,(BBC survey predicts BJP majority in Karnataka...,"[('2023-04-25 12:07:59+02', 'https://newsmeter...",(Fake BBC survey predicts BJP majority in Karn...,eng,BBC survey predicts BJP majority in Karnataka ...,61
132897,(Tipps eines italienischen Arztes zum Coronavi...,"[(1585932655.0, 'https://correctiv.org/faktenc...",(Kettenbrief: Angebliche Tipps eines italienis...,deu,Tipps eines italienischen Arztes zum Coronavirus,48
152951,(claims Fanta Lemon will be discontinued in Eu...,"[(1653941147.0, 'http://checkyourfact.com/2022...",(FACT CHECK: Is Fanta Lemon Being Discontinued...,eng,claims Fanta Lemon will be discontinued in Eur...,72
289922,(Photo shows a rally in support of the men acc...,"[('2023-07-30 12:42:10+02', 'https://www.booml...",(Unrelated Photo Shared As Rally In Support Of...,eng,Photo shows a rally in support of the men accu...,122
125531,(The CDC released data showing that 7 out of 1...,"[(1623916680.0, 'https://misbar.com/factcheck/...","(الصفحة الرئيسية, Homepage, [ara, 1.0])",eng,The CDC released data showing that 7 out of 10...,92


In [ ]:
tmp.iloc[4]['content']

'The CDC released data showing that 7 out of 10 Americans are declining the COVID-19 vaccine.'

In [ ]:
for x in tmp.index.to_list():
    posts = mapping_df[mapping_df['fact_check_id'] == x]['post_id'].to_list()
    print(f"for fact_check {x}: ")
    for p in posts_df.loc[posts].index.to_list():
        print(f"post: {posts_df.loc[p]['eng_content']}")
    print('summaries:')
    print(generate_summaries_in_batches(posts_df.loc[posts_df.loc[posts].index.to_list()]['content']))
    print("===================")

for fact_check 333674: 
summaries:


Processing Batches: 0it [00:00, ?it/s]


[]
for fact_check 132897: 
summaries:


Processing Batches: 0it [00:00, ?it/s]


[]
for fact_check 152951: 
summaries:


...

[]
for fact_check 289922: 
summaries:


Processing Batches: 0it [00:00, ?it/s]


[]
for fact_check 125531: 
summaries:


Processing Batches: 0it [00:00, ?it/s]

[]


In [ ]:
file_path = './data/summaries/Qwen2.5-7B-Instruct/fact_check_summaries.csv'

if os.path.exists(file_path):
    # Load the existing dataframe
    FC_summaries = pd.read_csv(file_path).set_index('fact_check_id')
    print("Dataframe loaded from file.")
else:
    # Create an empty dataframe with specific columns
    FC_summaries = pd.DataFrame(columns=['fact_check_id', 'summary']).set_index('fact_check_id')
    print("File not found. Created an empty dataframe.")

Dataframe loaded from file.


In [ ]:
FC_summaries

,summary
fact_check_id,
9136,Michel Temer exaggerated the time it took for ...
100280,"Pope Francis, along with other religious leade..."
9135,Michel Temer claimed his government approved a...
36946,"Senators debated the 2016 project 131/2015, wh..."
63469,The government's roadmap to ease COVID-19 rest...
...,...
327517,"Presidents of the European Parliament, Europea..."
329521,"In 10 years, Ukrainian schools will produce a ..."
316047,"Speculations about intercepted conversations, ..."


In [ ]:
fc_to_summarize = fc_sorted.loc[~fc_sorted.index.isin(FC_summaries.index)].sort_values(by='text_length', ascending=False)
fc_to_summarize

,claim,instances,title,language,content,text_length
fact_check_id,,,,,,
40048,"(Conservative commentator Ben Shapiro tweeted,...","[(1631054280.0, 'https://misbar.com/factcheck/...",,eng,"Conservative commentator Ben Shapiro tweeted, ...",275
174739,(معلم في السعودية سُجن على ذمة 100 ألف ريال لم...,"[(1626137580.0, 'https://misbar.com/factcheck/...",,ara,معلم في السعودية سُجن على ذمة 100 ألف ريال لم ...,275
173690,(كانت سيارة هونـدا أكورد الجيل ٨ على بعد ٥٠٠ م...,"[(1597919760.0, 'https://misbar.com/factcheck/...",,ara,كانت سيارة هونـدا أكورد الجيل ٨ على بعد ٥٠٠ م ...,275
27059,(As prefeituras regionais vão substituir as su...,"[(1484611140.0, 'https://piaui.folha.uol.com.b...",,por,As prefeituras regionais vão substituir as sub...,275
173494,"(قصة رواها بيل غيتس عندما سُئل ""هل هناك شخص أغ...","[(1580731320.0, 'https://misbar.com/factcheck/...",,ara,"قصة رواها بيل غيتس عندما سُئل ""هل هناك شخص أغن...",274
...,...,...,...,...,...,...
22432,"(Akun IDX di Telegram, IDX account on Telegram...","[(1611273540.0, 'https://cekfakta.com/focus/61...","([SALAH] Akun IDX di Telegram, [FALSE] IDX acc...",msa,Akun IDX di Telegram,20
199089,"(“SITUBONDO LOCKDOWN”, “SITUBONDO LOCKDOWN”, [...","[(1586303880.0, 'https://cekfakta.com/focus/48...","([SALAH] “SITUBONDO LOCKDOWN”, [FALSE] “SITUBO...",msa,“SITUBONDO LOCKDOWN”,20
59688,"(Foto “Pesinden 2024”, Photo “Pesinden 2024”, ...","[(1595635080.0, 'https://cekfakta.com/focus/44...","([SALAH] Foto “Pesinden 2024”, [FALSE] Photo “...",msa,Foto “Pesinden 2024”,20


In [ ]:
group_number = 0
group_size = 100

while True:
    frame = fc_to_summarize.iloc[group_number * group_size : (group_number + 1) * group_size]

    summaries = generate_summaries_in_batches(frame['content'].to_list(), batch_size=16)
    indices = frame.index.astype(int).to_list()

    # Combine new lists into a dataframe
    new_rows = pd.DataFrame({'fact_check_id': indices, 'summary': summaries}).set_index('fact_check_id')

    # Append new rows to the original dataframe
    FC_summaries = pd.concat([FC_summaries, new_rows])

    #save summaries
    FC_summaries.to_csv(file_path)

    print(f"fact_checks from {group_number * group_size} to {(group_number + 1) * group_size} has been summarized!")
    group_number += 1

Processing Batches:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Processing Batches: 100%|██████████| 7/7 [04:50<00:00, 41.55s/it]


fact_checks from 0 to 100 has been summarized!


Processing Batches:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Processing Batches: 100%|██████████| 7/7 [03:11<00:00, 27.34s/it]


fact_checks from 100 to 200 has been summarized!


Processing Batches:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Processing Batches: 100%|██████████| 7/7 [03:24<00:00, 29.25s/it]


...

Processing Batches:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Processing Batches: 100%|██████████| 7/7 [02:48<00:00, 24.14s/it]


fact_checks from 3200 to 3300 has been summarized!


Processing Batches:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Processing Batches: 100%|██████████| 7/7 [03:30<00:00, 30.00s/it]


fact_checks from 3300 to 3400 has been summarized!


Processing Batches:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


## Summarize posts

In [24]:
file_path = './data/summaries/Qwen2.5-7B-Instruct/posts_summaries.csv'

if os.path.exists(file_path):
    # Load the existing dataframe
    posts_summaries = pd.read_csv(file_path).set_index('post_id')
    print("Dataframe loaded from file.")
else:
    # Create an empty dataframe with specific columns
    posts_summaries = pd.DataFrame(columns=['post_id', 'summary']).set_index('post_id')
    print("File not found. Created an empty dataframe.")

Dataframe loaded from file.


In [25]:
posts_summaries

,summary
post_id,
21013,Prolonged use of masks can lead to hypercapnia...
5925,Joseph C De Gregorio and Tina M Brown pledge n...
15231,"The Mexican government plans to spend 5,792,11..."
26071,"On May 30, 2021, in Yongin Suji, videos claimi..."
27199,Weakened blood vessels in the head can rupture...
...,...
9720,"While humanity is in quarantine, Planet Earth ..."
3951,Painful scenes of oppression and suffering on ...
24262,The court has asked to seal the Shivling found...


In [ ]:
dev_posts_ids = []
dev_posts_ids.extend(tasks['crosslingual']['posts_dev'])
for lang in tasks['monolingual'].keys():
    dev_posts_ids.extend(tasks['monolingual'][lang]['posts_dev'])

In [ ]:
len(dev_posts_ids)

2443

In [ ]:
posts_sorted.loc[18820]

,18820
ocr,
text,Sé lo que son informes epidemiológicos; los do...
content,Sé lo que son informes epidemiológicos; los do...
eng_content,I know what epidemiological reports are; the t...
language,spa
text_length,19851


In [16]:
# posts_to_summarize = posts_sorted.loc[dev_posts_ids].sort_values(by='text_length', ascending=False)
posts_to_summarize = posts_sorted.loc[list(set(mapping_df['post_id'].to_list()))].sort_values(by='text_length', ascending=False)
posts_to_summarize

,ocr,text,content,eng_content,language,text_length
post_id,,,,,,
25778,AFTONAL LIST OF AVERE EVENTS OF SPECIAL p Pfiz...,Neun Seiten Nebenwirkungen: Komplette Auflistu...,AFTONAL LIST OF AVERE EVENTS OF SPECIAL p Pfiz...,AFTONAL LIST OF AVERE EVENTS OF SPECIAL p Pfiz...,deu,32243
14526,"hypothyroidism; Autoimmune er ear disease,Auto...",While they are distracted with Ukraine and Rus...,"hypothyroidism; Autoimmune er ear disease,Auto...","hypothyroidism; Autoimmune er ear disease,Auto...",eng,24760
26466,sina Sina Finance Hong Kong Stocks > Hong Kong...,"The cat crying and the mouse fake mercy, the g...",sina Sina Finance Hong Kong Stocks > Hong Kong...,sina Sina Finance Hong Kong Stocks > Hong Kong...,eng,22249
24616,Covid-190 home Mr Gree100 - - - Besli sadiste ...,Corona beta is put on the streets as their own...,Covid-190 home Mr Gree100 - - - Besli sadiste ...,Covid-190 home Mr Gree100 - - - Besli sadiste ...,eng,20770
27482,090177e196ea1800\Approved Approved On: 30-Apr-...,A confidential Pfizer report has been leaked. ...,090177e196ea1800\Approved Approved On: 30-Apr-...,090177e196ea1800\Approved Approved On: 30-Apr-...,eng,16040
...,...,...,...,...,...,...
21853,,,,,eng,0
21852,,,,,eng,0
27769,,,,,eng,0


In [17]:
posts_to_summarize = posts_to_summarize.loc[~posts_to_summarize.index.isin(posts_summaries.index)]
posts_to_summarize

,ocr,text,content,eng_content,language,text_length
post_id,,,,,,
25778,AFTONAL LIST OF AVERE EVENTS OF SPECIAL p Pfiz...,Neun Seiten Nebenwirkungen: Komplette Auflistu...,AFTONAL LIST OF AVERE EVENTS OF SPECIAL p Pfiz...,AFTONAL LIST OF AVERE EVENTS OF SPECIAL p Pfiz...,deu,32243
14526,"hypothyroidism; Autoimmune er ear disease,Auto...",While they are distracted with Ukraine and Rus...,"hypothyroidism; Autoimmune er ear disease,Auto...","hypothyroidism; Autoimmune er ear disease,Auto...",eng,24760
26466,sina Sina Finance Hong Kong Stocks > Hong Kong...,"The cat crying and the mouse fake mercy, the g...",sina Sina Finance Hong Kong Stocks > Hong Kong...,sina Sina Finance Hong Kong Stocks > Hong Kong...,eng,22249
24616,Covid-190 home Mr Gree100 - - - Besli sadiste ...,Corona beta is put on the streets as their own...,Covid-190 home Mr Gree100 - - - Besli sadiste ...,Covid-190 home Mr Gree100 - - - Besli sadiste ...,eng,20770
27482,090177e196ea1800\Approved Approved On: 30-Apr-...,A confidential Pfizer report has been leaked. ...,090177e196ea1800\Approved Approved On: 30-Apr-...,090177e196ea1800\Approved Approved On: 30-Apr-...,eng,16040
...,...,...,...,...,...,...
21853,,,,,eng,0
21852,,,,,eng,0
27769,,,,,eng,0


In [26]:
posts_to_summarize = posts_to_summarize.iloc[3000:20000].sample(12000 - len(posts_summaries))
posts_to_summarize

,ocr,text,content,eng_content,language,text_length
post_id,,,,,,


In [22]:
posts_to_summarize.sort_values(by='text_length', ascending=False)

,ocr,text,content,eng_content,language,text_length
post_id,,,,,,
10472,,Fascinating Timeless! History repeats itself....,Fascinating Timeless! History repeats itself....,Fascinating Timeless! History repeats itself....,eng,880
25874,,URGENT À FAIRE CIRCULER Il faut absolument fa...,URGENT À FAIRE CIRCULER Il faut absolument fa...,URGENT TO CIRCULATE It is absolutely necessar...,fra,817
27934,"C.R.T BALASO THANK YOU, FRONTLINERS. MON-LEYNA...",IN A FEW MONTHS This will be the cover of Time...,"C.R.T BALASO THANK YOU, FRONTLINERS. MON-LEYNA...","C.R.T BALASO THANK YOU, FRONTLINERS. MON-LEYNA...",eng,808
16359,great barrier 2009 great barrier 2019 Amazonia...,Partly False Information: Incorrectly dated ph...,great barrier 2009 great barrier 2019 Amazonia...,great barrier 2009 great barrier 2019 Amazonia...,eng,784
21690,,Zimbabwe Court Sentences Gay Couple to One Yea...,Zimbabwe Court Sentences Gay Couple to One Yea...,Zimbabwe Court Sentences Gay Couple to One Yea...,eng,755
...,...,...,...,...,...,...
8079,T The Sunnah of God --- SE IN 11,THE PORT OF TRIESTE ON 10/15/2021,T The Sunnah of God --- SE IN 11THE PORT OF TR...,T The Sunnah of God --- SE IN 11THE PORT OF TR...,eng,65
7567,,China installed this banner in Ladakh. Clear m...,China installed this banner in Ladakh. Clear m...,China installed this banner in Ladakh. Clear m...,eng,61
11944,,Impresionante derrumbe en el cañón del sumidero,Impresionante derrumbe en el cañón del sumidero,Impressive collapse in the sinkhole canyon,spa,47


In [23]:
group_number = 0
group_size = 100

while True:
    if group_number * group_size > len(posts_to_summarize):
        break

    frame = posts_to_summarize.iloc[group_number * group_size : (group_number + 1) * group_size]

    summaries = generate_summaries_in_batches(frame['eng_content'].to_list(), batch_size=8)
    indices = frame.index.astype(int).to_list()

    # Combine new lists into a dataframe
    new_rows = pd.DataFrame({'post_id': indices, 'summary': summaries}).set_index('post_id')

    # Append new rows to the original dataframe
    posts_summaries = pd.concat([posts_summaries, new_rows])

    #save summaries
    posts_summaries.to_csv(file_path)

    print(f"posts from {group_number * group_size} to {(group_number + 1) * group_size} has been summarized!")
    group_number += 1

Processing Batches:   0%|          | 0/13 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Processing Batches: 100%|██████████| 13/13 [04:04<00:00, 18.80s/it]


posts from 0 to 100 has been summarized!


Processing Batches:   0%|          | 0/9 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Processing Batches: 100%|██████████| 9/9 [02:49<00:00, 18.80s/it]

posts from 100 to 200 has been summarized!


In [ ]:
# prompt: how many row in summary column of posts_summaries have the word "**Summary:**"? delete the "**Summary:**" from them.

import pandas as pd

# Load the dataframe
file_path = './data/cleaned data/posts_summaries.csv'
posts_summaries = pd.read_csv(file_path).set_index('post_id')

# Count rows with "**Summary:**" in the 'summary' column
count = posts_summaries['summary'].str.contains(r"\*\*Summary:\*\*", regex=True).sum()

print(f"Number of rows with '**Summary:**' in the 'summary' column: {count}")

# Remove "**Summary:**" from the 'summary' column
posts_summaries['summary'] = posts_summaries['summary'].str.replace(r"\*\*Summary:\*\*", "", regex=True)
posts_summaries['summary'] = posts_summaries['summary'].str.replace(r"### Summary:", "", regex=True)


# Save the updated dataframe
# posts_summaries.to_csv('./data/cleaned data/posts_summaries.csv')

Number of rows with '**Summary:**' in the 'summary' column: 109


In [ ]:
posts_summaries[posts_summaries['summary'].str.contains(r"Summary", regex=True)]

,summary
post_id,


In [ ]:
posts_summaries.to_csv('./data/cleaned data/posts_summaries.csv')

# LLaMA 7B

In [ ]:
!pip install transformers bitsandbytes accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "huggyllama/llama-7b"
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        quantization_config=quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
input_text = """
Artificial intelligence (AI) refers to the simulation of human intelligence in machines
that are programmed to think and act like humans. The term may also be applied to any
machine that exhibits traits associated with a human mind such as learning and problem-solving.
"""

text = """
title: '  ' | explanations: ' Are avocados good for you? '
"""

# Define the summarization prompt
prompt = f"""
        "Summarize the following text in between 10 to 30 words while preserving critical information, such as names, places, cities, dates, and key events.
        Exclude any details about the writer, including their emotions, opinions, gender, or other personal characteristics. Ensure the summary is concise, factual, and focused only on the main points.
        Avoid adding any information not present in the original text. Here's the text:
        "{text}"


        ````
        Summary:
        ````
        """
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    inputs["input_ids"],
    max_length=100,          # Limit the output length
    num_beams=5,             # Use beam search for better results
    repetition_penalty=2.0,  # Penalize repetition
    no_repeat_ngram_size=3,  # Avoid repeating phrases
    early_stopping=True      # Stop once a complete summary is generated
)

# Decode the output
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

Summarize the following text in one or two sentences:


title: '  ' | explanations: ' Are avocados good for you? '


Summary:
    Avocados are a great source of healthy fats, vitamins, and minerals.

"""

from __future__ import unicode_literals

import re
import unittest
from collections import OrderedDict
from io import String


In [ ]:
text = """
title: '  ' | explanations: ' Are avocados good for you? '
"""

In [ ]:
prompt = f"""
        "Summarize the following text in between 10 to 30 words while preserving critical information, such as names, places, cities, dates, and key events.
        Exclude any details about the writer, including their emotions, opinions, gender, or other personal characteristics. Ensure the summary is concise, factual, and focused only on the main points.
        Avoid adding any information not present in the original text. Here's the text:
        "{text}"
        """
messages = [
    {"role": "system", "content": "You are a precise summarization tool that transforms social media content into concise, fact-focused summaries. Your objective is to maximize information density while maintaining perfect clarity."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=50
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

['\n\n\n        "\n        "\n        "\n        "\n        "\n        "\n        "\n        "\n        "\n        "\n        "\n        "\n        "\n        "\n        "\n        "']

# Llama-2-7b-samsum

In [ ]:
!pip install transformers bitsandbytes accelerate

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id =  "SalmanFaroz/Llama-2-7b-samsum"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [ ]:
text = """FINISHED THE FEAR Anti-Covid measures in schools no longer make sense Wearing a mask and other anti-Covid measures (freezing, quarantine, tests) are not applicable. Studies and scientific facts* prove it: • THE EPIDEMIC is over in Belgium (according to data as of September 20, 2020) • The VIRUS has lost its virulence and is not transmitted by young people . The COVID-19 disease hardly affects young people • DEATHS due to Covid-19 have been recorded mainly among people who were already frail (comorbidities) • The MASK can cause physical and psychological sequelae serious, and it undermines the quality of education • Excessive HYGIENE destroys the good bacteria in the hands • PCR TESTS are too sensitive (many false positives) • STATISTICS are misleading (positive case sick or hospitalized) Parents Teachers Join our group of Parents2021 Parents2021 Register on our website:  Together we are STRONGER the sources are available on our website"""

In [ ]:
# Define the input prompt
text = """FINISHED THE FEAR Anti-Covid measures in schools no longer make sense Wearing a mask and other anti-Covid measures (freezing, quarantine, tests) are not applicable. Studies and scientific facts* prove it: • THE EPIDEMIC is over in Belgium (according to data as of September 20, 2020) • The VIRUS has lost its virulence and is not transmitted by young people . The COVID-19 disease hardly affects young people • DEATHS due to Covid-19 have been recorded mainly among people who were already frail (comorbidities) • The MASK can cause physical and psychological sequelae serious, and it undermines the quality of education • Excessive HYGIENE destroys the good bacteria in the hands • PCR TESTS are too sensitive (many false positives) • STATISTICS are misleading (positive case sick or hospitalized) Parents Teachers Join our group of Parents2021 Parents2021 Register on our website:  Together we are STRONGER the sources are available on our website"""
prompt = """
        Summarize the following text in between 10 to 30 words while preserving critical information, such as names, places, cities, dates, and key events.
        Exclude any details about the writer, including their emotions, opinions, gender, or other personal characteristics. Ensure the summary is concise, factual, and focused only on the main points.
        Avoid adding any information not present in the original text.

        ### Input:
        {text}

        ### Summary:
        """

inputs = tokenizer(prompt.format(text=text), return_tensors='pt').to(model.device)
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=100,
    )[0],
    skip_special_tokens=True
)

print("Output:",output)

Output: 
        Summarize the following text in between 10 to 30 words while preserving critical information, such as names, places, cities, dates, and key events.
        Exclude any details about the writer, including their emotions, opinions, gender, or other personal characteristics. Ensure the summary is concise, factual, and focused only on the main points.
        Avoid adding any information not present in the original text.

        ### Input:
        FINISHED THE FEAR Anti-Covid measures in schools no longer make sense Wearing a mask and other anti-Covid measures (freezing, quarantine, tests) are not applicable. Studies and scientific facts* prove it: • THE EPIDEMIC is over in Belgium (according to data as of September 20, 2020) • The VIRUS has lost its virulence and is not transmitted by young people . The COVID-19 disease hardly affects young people • DEATHS due to Covid-19 have been recorded mainly among people who were already frail (comorbidities) • The MASK can cause p

# Marcoroni-7B-v3-AWQ

In [ ]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB

In [ ]:
from vllm import LLM, SamplingParams

llm = LLM(model="TheBloke/Marcoroni-7B-v3-AWQ", quantization="awq", dtype="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 01-27 15:26:59 config.py:510] This model supports multiple tasks: {'score', 'embed', 'generate', 'classify', 'reward'}. Defaulting to 'generate'.
WARNING 01-27 15:27:00 config.py:588] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 01-27 15:27:00 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='TheBloke/Marcoroni-7B-v3-AWQ', speculative_config=None, tokenizer='TheBloke/Marcoroni-7B-v3-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=Obse

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

INFO 01-27 15:27:01 selector.py:217] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 01-27 15:27:01 selector.py:129] Using XFormers backend.
INFO 01-27 15:27:01 model_runner.py:1094] Starting to load model TheBloke/Marcoroni-7B-v3-AWQ...
INFO 01-27 15:27:02 weight_utils.py:251] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

INFO 01-27 15:28:41 weight_utils.py:296] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 01-27 15:28:43 model_runner.py:1099] Loading model weights took 3.8814 GB
INFO 01-27 15:29:01 worker.py:241] Memory profiling takes 18.47 seconds
INFO 01-27 15:29:01 worker.py:241] the current vLLM instance can use total_gpu_memory (14.75GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 01-27 15:29:01 worker.py:241] model weights take 3.88GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 3.38GiB; the rest of the memory reserved for KV Cache is 5.95GiB.
INFO 01-27 15:29:02 gpu_executor.py:76] # GPU blocks: 3045, # CPU blocks: 2048
INFO 01-27 15:29:02 gpu_executor.py:80] Maximum concurrency for 32768 tokens per request: 1.49x
INFO 01-27 15:29:06 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilizat

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:45<00:00,  1.29s/it]

INFO 01-27 15:29:51 model_runner.py:1535] Graph capturing finished in 45 secs, took 1.39 GiB
INFO 01-27 15:29:51 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 68.02 seconds


In [ ]:
text = """FINISHED THE FEAR Anti-Covid measures in schools no longer make sense Wearing a mask and other anti-Covid measures (freezing, quarantine, tests) are not applicable. Studies and scientific facts* prove it: • THE EPIDEMIC is over in Belgium (according to data as of September 20, 2020) • The VIRUS has lost its virulence and is not transmitted by young people . The COVID-19 disease hardly affects young people • DEATHS due to Covid-19 have been recorded mainly among people who were already frail (comorbidities) • The MASK can cause physical and psychological sequelae serious, and it undermines the quality of education • Excessive HYGIENE destroys the good bacteria in the hands • PCR TESTS are too sensitive (many false positives) • STATISTICS are misleading (positive case sick or hospitalized) Parents Teachers Join our group of Parents2021 Parents2021 Register on our website:  Together we are STRONGER the sources are available on our website"""
prompt = """
        Summarize the following text in between 10 to 30 words while preserving critical information, such as names, places, cities, dates, and key events.
        Exclude any details about the writer, including their emotions, opinions, gender, or other personal characteristics. Ensure the summary is concise, factual, and focused only on the main points.
        Avoid adding any information not present in the original text.

        ### Input:
        {text}

        ### Summary:
        """

prompts = [
    "Tell me about AI",
    "Write a story about llamas",
    "What is 291 - 150?",
    "How much wood would a woodchuck chuck if a woodchuck could chuck wood?",
]
prompt_template='''### Instruction:
{prompt}

### Response:
'''

prompts = [prompt.format(text=text)]

sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"\nPrompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.03s/it, est. speed input: 347.14 toks/s, output: 15.60 toks/s]


Prompt: '\n        Summarize the following text in between 10 to 30 words while preserving critical information, such as names, places, cities, dates, and key events.\n        Exclude any details about the writer, including their emotions, opinions, gender, or other personal characteristics. Ensure the summary is concise, factual, and focused only on the main points.\n        Avoid adding any information not present in the original text.\n\n        ### Input:\n        FINISHED THE FEAR Anti-Covid measures in schools no longer make sense Wearing a mask and other anti-Covid measures (freezing, quarantine, tests) are not applicable. Studies and scientific facts* prove it: • THE EPIDEMIC is over in Belgium (according to data as of September 20, 2020) • The VIRUS has lost its virulence and is not transmitted by young people . The COVID-19 disease hardly affects young people • DEATHS due to Covid-19 have been recorded mainly among people who were already frail (comorbidities) • The MASK can

# Mistral-7B-Instruct-v0.1

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

chat = [
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
  {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

tokenizer.apply_chat_template(chat, tokenize=False)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        quantization_config=quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1.
401 Client Error. (Request ID: Root=1-6797a7ec-08fe950e0da94b2b160a41a8;f2b2d3e6-c000-4b97-98d5-501c5b5ed362)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.

# DeepSeek-R1-Distill-Qwen-7B

In [ ]:
!pip install transformers bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype="auto",
        quantization_config=quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/6.62G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
text = """FINISHED THE FEAR Anti-Covid measures in schools no longer make sense Wearing a mask and other anti-Covid measures (freezing, quarantine, tests) are not applicable. Studies and scientific facts* prove it: • THE EPIDEMIC is over in Belgium (according to data as of September 20, 2020) • The VIRUS has lost its virulence and is not transmitted by young people . The COVID-19 disease hardly affects young people • DEATHS due to Covid-19 have been recorded mainly among people who were already frail (comorbidities) • The MASK can cause physical and psychological sequelae serious, and it undermines the quality of education • Excessive HYGIENE destroys the good bacteria in the hands • PCR TESTS are too sensitive (many false positives) • STATISTICS are misleading (positive case sick or hospitalized) Parents Teachers Join our group of Parents2021 Parents2021 Register on our website:  Together we are STRONGER the sources are available on our website"""

In [ ]:
prompt = f"""
            **Summarize the following social media text into a concise and factual summary. Your goal is to capture the essential information, focusing on names, locations, dates, events, and numerical data while omitting unnecessary details such as emotions, personal opinions, and social media-specific elements. Ensure the summary is as brief and precise as possible.**

            ---

            ### **Guidelines:**

            #### **Include**:
            - Names of people, organizations, and landmarks.
            - Locations (cities, countries, buildings, etc.).
            - Dates, times, and chronological order if multiple events.
            - Numerical data (e.g., statistics, monetary values, timelines).
            - Professional titles, projects, and event details.
            - Direct quotes **only if critical** to the main information.

            #### **Exclude**:
            - Descriptive language, emotions, or personal opinions.
            - Social media-specific elements like hashtags, likes, mentions, and emojis.
            - Background or contextual information unless essential.
            - Gender-specific references unless directly relevant.

            #### **Format**:
            1. Use active voice.
            2. Present factual information only.
            3. Keep the summary to a **single sentence (preferred)** or **maximum of 30 words**.
            4. Prioritize the most critical detail first.
            5. Maintain clarity and chronological order if multiple events are mentioned.

            ---

            ### **Task**:
            Now, summarize the following social media text according to these guidelines and only output the summary without any other description:

            **{text}**

            ### **Summary:**
"""
messages = [
    {"role": "system", "content": "You are a precise summarization tool that transforms social media content into concise, fact-focused summaries. Your objective is to maximize information density while maintaining perfect clarity."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=128
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


"<think>\nOkay, so I have to summarize this social media post into a concise and factual summary. Let me read through the text carefully.\n\nThe post starts by talking about anti-COVID measures in schools not making sense anymore. It mentions wearing masks and other measures like freezing, quarantine, and tests aren't applicable. Then it goes on to cite some studies and scientific facts. \n\nFirst, it says the epidemic is over in Belgium as of September 20, 2020. That's a key point. Next, the virus has lost its virulence and doesn't transmit well among young people. So, the COVID"